# Purpose
After filtering out the spammers, we can now work on the weibo data. The goal of this purpose is to extract hashtag and record the message and its corresponding hashtag. Then we will use word2vec to train the data.

We first store all messages that contain "#" to a list since hashtags in weibo is denoted as sth like #机器学习#. 
From that list, we extract hashtag(s) from each message and save it to a list. We record the message id and its corresponding hashtag in the list called hashtags. 

In [1]:
import csv
import pandas as pd
import numpy as np
import gensim # word2vec package
import jieba  # for Chinese text segementation

C:\Anaconda2\envs\gl-env\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('normal_all2')
print len(df)
df.head()

892539


,hashtag_count,mention_count,text,time,url_count,userid
0,0,0,冷言冷语。。。继续。。。这样很好。。。,2012-09-03 00:03:19,0,uWWBCGJE4
1,0,1,傻子 过去就过去了 越想只会让自己 越来越TM看不起自己 恨自己不是吗？有什么了不起的 人不...,2012-09-03 00:04:12,0,u1O52TEBM
2,0,0,闲来无事，想破人QQ空间密码，于是点击了一个美女的空间，提问：“请问男生最喜欢什么花！”我暗...,2012-09-03 00:02:10,0,u0AGHHPXK
3,0,0,北京今天，泛起秋意。,2012-09-03 00:07:36,0,uVGJLC4S5
4,0,2,@_@,2012-09-03 02:09:32,0,uATZPTH53


In [3]:
texts = df['text'].tolist()
print texts[0]

冷言冷语。。。继续。。。这样很好。。。


In [4]:
hashtag_text= []
for text in texts:
    try:  #handle exception when text is null
        if "#" in text:      
            hashtag_text.append(text)
    except:
        pass
print len(hashtag_text)

23353


In [5]:
hashtag_text_dict = {}   # key is the weibo, value is the unique id of weibo
i = 0
for text in texts:
    try:  #handle exception when text is null
        if "#" in text:      
            hashtag_text_dict[text] = i
    except:
        pass    
    i += 1
print len(hashtag_text_dict)

21005


In [81]:
"""""
print hashtag_text[0]
a=hashtag_text[0].index('#')
b=hashtag_text[0].index('#', hashtag_text[0].index('#')+1)
print hashtag_text[0][a+1: b]
print hashtag_text[0][b+1:]
try:
    hashtag_text[0][b+1:].index('#')
except:
    pass

今天以“周末能否约到我吃饭”来判别周末是否有F1的同学-----发现吃药了。啊哈哈哈哈 #晚安#
晚安



In [8]:
"""""
hashtags = []
i = 0
for text in hashtag_text_dict.keys():
    count = 0
    while count < 10:
            start = text.index('#')
            if text[start+1:].find('#')<0:
                break  
            else:
                end = text.index('#', start+1)
                hashtag = text[start+1: end]    #   #idea # is not a hashtag; #idea# is
                if ' ' not in hashtag:
                    hashtags.append((i, hashtag))
                text = text[end+1:]
                count += 1
                if text.find('#')<0:
                    break
    i += 1  

In [6]:
hashtags = []
for text, i in hashtag_text_dict.iteritems():
    count = 0
    while count < 10:
        start = text.find('#')
        if text[start+1:].find('#') < 0:
            break  
        else:
            end = text.find('#', start+1)
            hashtag = text[start+1: end]    #   #idea # is not a hashtag; #idea# is
            if ' ' not in hashtag:
                hashtags.append((i, hashtag))
            text = text[end+1:]
            count += 1
            if text.find('#')<0:
                break

In [7]:
print len(hashtags)

21654


In [8]:
for h in hashtags:
    print h[0], h[1]

889015 美图秀秀Android版
300331 筋肉人
300331 泥水妹
113822 微观点
433779 请回答1995
633618 Weico拼图
220073 Lifetrons
679704 后续
227650 你的能量让我盛放
748047 祝福
436752 TwoBebes早安问候
267590 反國民教育科
264710 校园迎新，翼起飞young
409145 日本药妆
233219 搜狐焦点开盘直播
611358 毛小辉
635871 靓车有道－旗舰店
862464 恋上古诗词
109404 中国好声音
553581 外星人大搜捕
124952 钙尔奇信赖体
435369 这只猫咪超可爱
334478 微博预告
334478 聚焦执法公正
600824 花旗銀行大廈
547398 征集您的见证语
68131 我爱小米手机
556367 大盛设计
115041 MTK版新浪微博
231827 宝马品牌
599627 金鹰节颁奖晚会
627933 午夜梦语
364550 面包树上的女人
74364 蓝海博龙国际大酒店宴会厅
74364 拼图
685622 张小盒心语
174059 大盛设计
802499 达人生日祝福
888856 美图秀秀iPhone版
886933 高级达人
419051 撤回國民教育科
733103 我是快乐的跑步控
92222 林峯
92222 林峰
92222 林峯好靚仔
92222 林峯好可愛
92222 正能量
92222 林峯我撐你
92222 天氣會暖d....太陽會亮d....
840937 心灵鸡汤
881341 一汽丰田成都车展
707707 巢祝福
113327 志愿关注
313956 武艺
247011 苏荷星闻
309751 上海东锦江希尔顿逸林酒店
538599 台湾行加油kris
389798 美图秀秀iPhone版
335965 收藏寻城记
299817 每日一车
121080 粉丝颁奖典礼
852463 方特感恩教师节
262563 携程全球酒店体验员
772216 周渝民
33747 南方公园
690927 我的唯一931
284100 中国好声音
374557 天津市,河西区,环湖中道
354922 童话二分之一
410099 GIF
14561 早安
579443 WeicoP

In [9]:
hashtag_dict = {}   # key is the unique weibo id, value is all hashtags it contains
for text, i in hashtag_text_dict.iteritems():
    hashtags2 = []
    count = 0
    while count < 10:
        start = text.index('#')
        if text[start+1:].find('#')<0:
            break  
        else:
            end = text.index('#', start+1)
            hashtag = text[start+1: end]    #   #idea # is not a hashtag; #idea# is
            if ' ' not in hashtag:
                hashtags2.append(hashtag)
            text = text[end+1:]
            count += 1
            if text.find('#')<0:
                break
    hashtag_dict[i] = hashtags2

In [11]:
for w in hashtag_dict[92222]:
    print w

林峯
林峰
林峯好靚仔
林峯好可愛
正能量
林峯我撐你
天氣會暖d....太陽會亮d....


In [12]:
ids = hashtag_text_dict.values() # a list of all weibo ids
oversea = [df['is_oversea'][i] for i in ids]

KeyError: 'is_oversea'

In [61]:
print oversea[90:100]

[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]


In [65]:
hashtag_text_dict.values()[93]

50066

In [69]:
for i in hashtag_dict[50066]:
    print i

第十六届中国国际投资贸易洽谈会


In [66]:
df[50066:50067]

,userid,text,hashtag_count,mention_count,url_count,is_oversea
50066,u0YUBS3TS,#第十六届中国国际投资贸易洽谈会#@ukn： @ukn： @ukn： @uYC3EJMUH：...,1,7,1,1


In [70]:
np.mean(oversea)

0.14606046179481075

In [75]:
hashtag_oversea = []
for i in range(len(oversea)):
    if oversea[i] == 1:
        hashtag_oversea.extend(hashtag_dict[hashtag_text_dict.values()[i]])
print len(hashtag_oversea)        

3206


In [76]:
for h in hashtag_oversea:
    print h

美图秀秀Android版
時間寵愛炎亞綸
炎亞綸給愛麗絲的奇蹟
炎亞綸
蓝海博龙国际大酒店宴会厅
拼图
台湾行加油kris
木瓜奶米糊
天津市,河西区,环湖中道
潘玮柏
杨丞琳
罗志祥
与狼共舞代言人
第十六届中国国际投资贸易洽谈会
靓车有道－旗舰店
香港科技大学图书馆
韩庚
愛情闖進門
拼图
9.8漳州信和活动
2pm4周年快樂
請插入話題名稱
中国好微博
石头门
美图GIF
拼图
便民服务
麦克风不停，越夜越动听
SB定律
老友記
李冰冰拒赴日本宣传
多啦A梦
怪叔叔跳古怪舞
中宇大厦
毕业那年
凌晨拍摄
NC粉无误
joongsim
joongsim
你在等什麼
汪东城
你在等什么
香港紅磡體育館
我在香港住
小女孩
午夜表情
时计堂
时计堂
2pm4周年快樂
拼图
交互设计体验日
潍院签到有惊喜
朝鲜版少女时代走红
北京青年
巢星指数
學民思潮
黑子的篮球
龍湖大廈
WeicoPlus
婚礼专业英语
尊活动
美图秀秀iPhone版
北京市,顺义区,一经路
图说零售
港漂感悟
2pm4周年快乐
Zoe做菜很用心
勝負之神
新宿駅・ＪＲ／山手線
穷屌丝晒图啦~~~
風之后
长江后浪
你们都去买X100吧
炎亞綸
007
小女孩
向上吧！少年
家有琴童
云南彝良贵州威宁交界地震
米饭宠你吃茄子
朴有天
勋鹿
好友印象
新城數碼巨聲音樂會
上海大学北门
咕咚运动
清唱团
无锡厂房
禾木村
LG大厦
袁来如诗
WeicoLomo
加大力度
深夜食糖
泉州医高专签到有惊喜
吳尊
新白发魔女传
5sing
美图秀秀Android版
桂语山房高级餐厅
咱也重走回青春
慧妍愛心傾城30年
superjunior出道2500天
superjunior出道2500天
期待炎亞綸新專輯
時間寵愛炎亞綸
谁也不许打击我不然我跟你没完
是有這麼缺內褲嗎
HIM雜誌
美图秀秀iPhone版
開學反洗腦
kkbox
黃石碼頭
攝影
微博科普
黄金海岸
四川凉山公益
Gif快手
“韩中歌谣祭"
羅力威
师兄师姐告诉你
耀安邨(馬鞍山路)
浙江省,杭州市,建德市,广兴北路
耿乐电子相册
卓文萱
金鹰大直播
本城事件有感而发
瓊華中心
晚安！
致美丽的你
实战经验
表哥传奇
红围脖免流量征集
灣仔政府大樓
香格里拉大酒店
胡夏
胡夏爱上台北
胡夏
爱上台北
美图秀秀Android版
我正在听
拼

In [77]:
hashtag_native = []
for i in range(len(oversea)):
    if oversea[i] == 0:
        hashtag_native.extend(hashtag_dict[hashtag_text_dict.values()[i]])
print len(hashtag_native)

18448


In [78]:
for h in hashtag_native:
    print h

特别声明
微观点
请回答1995
Weico拼图
面包树上的女人
后续
你的能量让我盛放
祝福
反國民教育科
校园迎新，翼起飞young
勛鹿一生推
关于新生
搜狐焦点开盘直播
毛小辉
明春苑
恋上古诗词
中国好声音
结婚进行式
钙尔奇信赖体
六年你就是美丽的
微博预告
聚焦执法公正
花旗銀行大廈
征集您的见证语
我爱小米手机
大盛设计
宝马品牌
吃货指南
聚焦投洽会
营销你、改变你、成就你
每日一喊
看巨变
大盛设计
达人生日祝福
美图秀秀iPhone版
瑞麒关爱行动
早安广州
《周末只听音乐》版宣
林峯
林峰
林峯好靚仔
林峯好可愛
正能量
林峯我撐你
天氣會暖d....太陽會亮d....
心灵鸡汤
中國愛大歌會
健康时光
志愿关注
推兔
體檢真可怕
苏荷星闻
上海东锦江希尔顿逸林酒店
美图秀秀iPhone版
收藏寻城记
大爱清尘
大爱清尘
粉丝颁奖典礼
方特感恩教师节
携程全球酒店体验员
世界真細小小小
我的唯一931
中国好声音
童话二分之一
GIF
2012网商大会
早安
整合企业的社交媒体文化
喜欢的话
美图秀秀iPhone版
古典吉他练习日记
家有颜小七
6日速效瘦身套组
纽斯桑拿会所
江湖儿女
微言大义
120902天生我才精彩视频
微博寻人
新白发魔女传
中国好声音
nichkhun出道4周年
宋茜出道三周年
fx出道三周年
nichkhun出道4周年
宋茜出道三周年
fx出道三周年
沃享中秋
上海邮政博物馆(北苏州路分馆)
蝙蝠侠抱枕
网博会
WeicoLomo
刀剑神域
bareminerals
丽江最美露美雅阁
锦里
炫音kt完美好声音
强哥侃营销
李晨爱送石头心被吐槽
兴亚建国饭店
馬國明
AVEN美肤
下午茶分享
咕咚运动
暮光之城
第69届威尼斯电影节
周杰伦
免费获得花园鞋中奖名单
烽火佳人
乱世佳人
李晨同款
潜规则
找骂贴
市长带你游
互相监督
长沙黄花机场
拼图
享乐会--当代艺术品鉴沙龙
PROMETHEUS
我要港人治港
名媛望族
吃货掌门人
青年南街
生活中的数学
理想国际大厦
一上一下
時間寵愛炎亞綸
期待炎亞綸新專輯
新浪博客七周年
神话
外星人大搜捕
美国留学
金融工程
電視劇非緣勿擾
美图秀秀iPhone版
北京领养
楊宗緯
港灣豪庭廣場(大角咀)
云南生活
全球汽车论坛
鹿晗
台州市黄岩区
再不疯狂我们就老了
关

In [81]:
for text in hashtag_text_dict.keys()[:5]:
    print text

#美图秀秀Android版#
#特别声明#据粉丝反应，最近又出现在地铁口打着《美丽俏佳人》专家之名的行骗者。该人为穿着鲜艳的男子，五短身材，带黑框眼镜。特此声明提醒各位观众若是节目邀请，不会收取任何费用，并会出示名片，请大家一定要警惕分辨，以防上当受骗哦！[酷]也在此感谢给我们提供信息的粉丝哦！
这么多啊？送钱给老师干嘛？他又不能把知识回赠给你？[疑问] //@ukn： 完全不同意此博说法，这个市场价真是太疯狂了，过个教师节家长要送1万？耸人听闻！ //@uB4H2BPLF：  #微观点#人人都送礼如同人人都没有送礼，给老师和医生送红包，就像给佛祖上贡品，都是期望得到一个心理上的安慰。
Hail to the redskins!! RT Ryan5O4: KDTrey5 Those Redskins going down tomorrow! #WHODAT
//@ukn： #请回答1995# 我的HOT，我的东方神起。


# Chinese text segementation

In [13]:
#remove hashtag from each message
text_rem_hash = []
for text, i in hashtag_text_dict.iteritems():
    for h in hashtags:
        if h[0] == i:
            text=text.replace(h[1], '')
    text_rem_hash.append(text)

In [192]:
"""""
print hashtag_text[36]
text = hashtag_text[36]
for h in hashtags:
    if h[0] == 36:
        text=text.replace(h[1], '')
print text

[心]//@ukn： #我喜欢lay#@ukn：  //@uMMSULRPD： 你也嘟嘴>< //@ukn： #我喜欢lay#//@ukn： #我喜欢lay# [ali扑倒]//@ukn：  有話要說嗎？#我喜欢lay#
[心]//@ukn： ##@ukn：  //@uMMSULRPD： 你也嘟嘴>< //@ukn： ##//@ukn： ## [ali扑倒]//@ukn：  有話要說嗎？##


In [14]:
print text_rem_hash[36]
print text_rem_hash[17]

##不同风格的精彩力作一一呈现，感谢同事们的合作努力！也感谢客户信任！
昨天我洗完澡，觉得对不起它，所以今天也带小朋友来洗洗。@ukn： [爱你] 我在## http://t.cn/zWITkNh


In [249]:
"""""
text = text_rem_hash[49]
count = 0
while count <10:
    start = text.find('@')
    end = text.find('：')
    text = text[:start]+text[end+1:]
    if text.find('@') < 0:
        break
    count += 1
print text

##9月8日《芭莎艺术》作为首席合作媒体与 合作开展"The House of Art”艺术活动:其中包括大堂装置艺术展,艺术家讲座, 《亮点》展览及现场行为艺术表演, 贵宾之夜艺术派对等等精彩内容。明天在外滩18号，更多艺术的惊喜等你来发现！


In [271]:
"""""
text = '购书网址：当当：http://t.cn/zWDk4Se 亚马逊：http://t.cn/zWDkwmw 各地书店或亦有售。这本书每卖出一本，作为编者，我个人向@uONPVBDIJ： 捐赠十分钱。十分微薄，十分努力。艰难时世，与君共勉。'
text='今天奥巴马接受民主党提名演讲的中文翻译全文和视频。http://t.cn/zWDMKqC 看不了长微博的可看这个网址 http://t.cn/zWDKSnF,0,0,2'
count = 0
while count <10:
    start = text.find('http')
    if text.find(' ', start+1) < 0:  # url in the end
        text = text[:start]
    else:                            # url in the middle
        end = text.find(' ')  
        text = text[:start]+text[end+1:]
    if text.find('http') < 0:
        break
    count += 1
print text

今天奥巴马接受民主党提名演讲的中文翻译全文和视频。看不了长微博的可看这个网址 


In [15]:
# remove @mention
text_rem_at = []
for text in text_rem_hash:
    count = 0
    while count < 20:
        if text.find('@') < 0:
            break
        else:
            start = text.find('@')
            if text.find('：', start+1) < 0: # @ is at the end of the message
                text = text[:start]
                break
            else:
                end = text.find('：', start+1)
                text = text[:start]+text[end+3:]
        count += 1    
    text_rem_at.append(text)

In [ ]:
"""""
s = '：'
s = ' '
s
#s.find('：')
#s[3:]

In [16]:
print len(text_rem_hash)
print len(text_rem_at)
print text_rem_hash[7867]
print text_rem_at[7867]

21005
21005
就业！ //@uPA3J41TM：  ##广州本地萌猫子爱爱酱年龄约三个月，会用猫沙，会食猫糧，最喜欢打噜噜求抱抱。要求封窗封阳台，绝育，科学喂养，有病医病没病打疫苗，不放弃不抛弃。领养联系@ukn： [求关注]
就业！ //  ##广州本地萌猫子爱爱酱年龄约三个月，会用猫沙，会食猫糧，最喜欢打噜噜求抱抱。要求封窗封阳台，绝育，科学喂养，有病医病没病打疫苗，不放弃不抛弃。领养联系 [求关注]


In [17]:
# remove url
text_rem_url = []
for text in text_rem_at:
    count = 0
    while count < 10:
        if text.find('http') < 0:
            break  
        else:
            start = text.find('http')
            if text.find(' ', start+1) < 0:  # url in the end
                text = text[:start]
            else:                            # url in the middle
                end = text.find(' ')  
                text = text[:start]+text[end+1:]
                if text.find('http') < 0:
                    break
            count += 1
    text_rem_url.append(text)

In [19]:
print text_rem_url[17]
print hashtag_text_dict.keys()[17]
for i in hashtag_dict[hashtag_text_dict.values()[17]]:
    print i

昨天我洗完澡，觉得对不起它，所以今天也带小朋友来洗洗。 [爱你] 我在## 
昨天我洗完澡，觉得对不起它，所以今天也带小朋友来洗洗。@ukn： [爱你] 我在#靓车有道－旗舰店# http://t.cn/zWITkNh
靓车有道－旗舰店


In [24]:
# remove punctuation
punc = ['#', '，', '。', '！', '？', '：', '“', '”', '（', '）', '【', '】', '+', '、', '％', '.', ':', '(', ')', '[', ']', '《', '》', '', '；', '～', '＂']
punc+[',', '!', '?', '+', '=', '-', '~', '"', '<', '>', '/', '^', ';']
text_rem_punc = []
for text in text_rem_url:
    text = text.replace(' ', '')
    for c in punc:
        if c in text:
            text = text.replace(c, '')            
    text_rem_punc.append(text)

In [16]:
# remove Chinese punctuation
punc = ['#', '，', '。', '！', '？', '：', '“', '”', '（', '）', '【', '】', '+', '、', '％', '.', ':', '(', ')', '[', ']', '《', '》', '', '；', '～', '＂']
text_rem_punc = []
for text in text_rem_url:
    for c in punc:
        if c in text:
            text = text.replace(c, '')
    text_rem_punc.append(text)

In [25]:
print text_rem_punc[17]

昨天我洗完澡觉得对不起它所以今天也带小朋友来洗洗爱你我在


# Save to file for lucene indexing

In [27]:
# find text, time, hashtag, and userid
#hashtag_text_dict      key is the weibo, value is the unique id of weibo
df.head()

,hashtag_count,mention_count,text,time,url_count,userid
0,0,0,冷言冷语。。。继续。。。这样很好。。。,2012-09-03 00:03:19,0,uWWBCGJE4
1,0,1,傻子 过去就过去了 越想只会让自己 越来越TM看不起自己 恨自己不是吗？有什么了不起的 人不...,2012-09-03 00:04:12,0,u1O52TEBM
2,0,0,闲来无事，想破人QQ空间密码，于是点击了一个美女的空间，提问：“请问男生最喜欢什么花！”我暗...,2012-09-03 00:02:10,0,u0AGHHPXK
3,0,0,北京今天，泛起秋意。,2012-09-03 00:07:36,0,uVGJLC4S5
4,0,2,@_@,2012-09-03 02:09:32,0,uATZPTH53


In [28]:
count = 0
for t in hashtag_text_dict.keys():
    for index, row in df.iterrows():    
        if row['text'] == t:
            count += 1
print count

KeyboardInterrupt: 

In [ ]:
df2 = pd.DataFrame({'userid': userids, 'text': texts, 'hashtag': , 'time': times})

In [18]:
seg_list = []
for i in range(len(text_rem_punc)):
    words = jieba.cut(text_rem_punc[i], cut_all=False)
    seg_list.append(words)

In [19]:
# remove english punctuation
punc2 = set([',', '!', '?', '+', '=', '-', '~', '"', '<', '>', '/', '^'])
cleaned_list = []
for words in seg_list:
    words2 = []
    for w in words:
        if w not in punc2:
            words2.append(w)
    cleaned_list.append(words2)

Building prefix dict from the default dictionary ...
Loading model from cache c:\users\bao\appdata\local\temp\jieba.cache
Loading model cost 0.416 seconds.
Prefix dict has been built succesfully.


In [20]:
for w in cleaned_list[2000]:
    print w

 
 
20120827
娛樂
新聞
報道
 
想
看到
的
人
都
在
視頻
裏
不解
釋
了
自己
看
哈哈
 


In [21]:
seg_list2 = []
for i in range(len(cleaned_list)):
    words = [w for w in cleaned_list[i]]
    words.extend(hashtag_dict[hashtag_text_dict.values()[i]])
    seg_list2.append(words)

In [22]:
for w in seg_list2[15000]:
    print w

 
百万
上海
人游
宁波
今日
从
上海
出发
上海
市民
接受
记者
采访
他
说
我
知道
的
宁波
只是
知道
宁波
有个
雅戈尔
我
心里
有着
莫名
的
惆怅
 
 
这是
一个
年轻人
的
心声
宁波
旅游
任重道远
旅游达人游宁波


In [48]:
"""""
seg_list = []
for i in range(1000):
    words = jieba.lcut(text_rem_punc[i], cut_all=False)
    words.extend(hashtag_dict[hashtag_text_dict.values()[i]])
    seg_list.append(words)

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-48-128440533883>, line 6)

In [ ]:
"""""
seg_list = []
for i in range(len(text_rem_punc)):
    words = jieba.lcut(text_rem_punc[i], cut_all=False)
    words.extend(hashtag_dict[hashtag_text_dict.values()[i]])
    seg_list.append(words)

In [23]:
print len(seg_list2)

21005


In [455]:
"""""
print cleaned_text[4]
seg_list = jieba.cut(cleaned_text[4], cut_all=False)
print "/ ".join(seg_list)

   如果老兵们每天都在看微博 看国之将亡 可想要再次浴血奋战年 如果是 请不要为政党 请为人权 为产权 而战 产真是不能共之的 这是最大的罪恶根源   希望你们的鲜血没有白流
 /  /  / 如果/ 老兵/ 们/ 每天/ 都/ 在/ 看/ 微博/  / 看国/ 之/ 将/ 亡/  / 可/ 想要/ 再次/ 浴血奋战/ 年/  / 如果/ 是/  / 请/ 不要/ 为/ 政党/  / 请/ 为/ 人权/  / 为/ 产权/  / 而战/  / 产/ 真是/ 不能/ 共/ 之/ 的/  / 这是/ 最大/ 的/ 罪恶根源/  /  /  / 希望/ 你们/ 的/ 鲜血/ 没有/ 白流


# Use word2vec

In [109]:
"""""
q = [u'我到河北省来', u'好棒好棒哒']
z = [list(jieba.cut(i)) for i in q]
model = gensim.models.Word2Vec(z, min_count=1)
model.similar_by_word(u'我')

Building prefix dict from the default dictionary ...
Dumping model to file cache c:\users\bao\appdata\local\temp\jieba.cache
Loading model cost 1.626 seconds.
Prefix dict has been built succesfully.


[(u'\u54d2', 0.1395096778869629),
 (u'\u6765', 0.0669124498963356),
 (u'\u68d2', 0.04685476794838905),
 (u'\u597d', -0.08643458038568497),
 (u'\u5230', -0.09045886993408203),
 (u'\u6cb3\u5317\u7701', -0.10627628862857819)]

In [24]:
model = gensim.models.Word2Vec(seg_list2, size=100, window=5, min_count=1, workers=4)

In [50]:
for result in model.similar_by_word('林峯', topn = 30):
    print result[0], result[1]

林峰 0.976476430893
仔好 0.892298817635
好靚 0.888503670692
可愛我撐 0.884218096733
給力 0.866235136986
潜行狙击 0.853998064995
愛你 0.847748994827
期待炎亞綸新專輯 0.837695121765
花心 0.83590310812
親親 0.835597753525
蕭敬騰 0.834171891212
巨星靚聲音樂會 0.834097921848
林峯好靚仔 0.832183837891
心心 0.820528864861
LOLLIPOP 0.819624483585
新白发魔女传 0.816839396954
圍觀 0.80558347702
F 0.80365896225
韩庚 0.802310824394
美图秀秀iPhone版 0.795763731003
帥哥 0.789156973362
巨聲靚聲音樂會 0.789082705975
林峯好可愛 0.786630749702
吳尊 0.785300970078
你在等什麼 0.777880370617
live 0.775707125664
美图秀秀Android版 0.775393784046
韩庚世界巡演泰国站 0.775160551071
jamcast 0.772534966469
汪东城 0.771771907806


In [51]:
for result in model.similar_by_word('国民教育', topn = 30):
    print result[0], result[1]

酒后 0.866746902466
档位 0.841793000698
坏小孩 0.839899301529
环塔 0.837116956711
甜甜 0.83686029911
挨饿 0.835853159428
划入 0.835664391518
Sportsport 0.834870457649
使油 0.829960286617
鹿 0.826851248741
负荷 0.826632857323
120906 0.82624912262
涛声 0.824701547623
浙江电信 0.823918998241
这个励志故事的结尾太霸气了 0.821348071098
撞见 0.821168243885
日落而息 0.818288326263
羊羹 0.818276524544
天长日久 0.818090200424
³ 0.817494273186
自提 0.816175103188
扬花 0.816169559956
抽筋 0.816119253635
偷袭 0.815959215164
提防 0.815850675106
试着 0.815618038177
秀丽 0.815150141716
TOUCH 0.815052390099
﹖ 0.81474584341
压寨夫人 0.814022779465


In [47]:
test_hashtag = []
for h in hashtags:
    if h[1] == '国民教育':
        print h[0]
        test_hashtag.append(h[0])

553714
390510
281512


In [48]:
for text, i in hashtag_text_dict.iteritems():
    if i in set(test_hashtag):
        print text

#国民教育# 我突然觉得香港人反对搞这个是应该的.... 真的会教坏小孩...
#国民教育# 香港政府推行国民教育，提供了一个充斥洗脑性质的教学指引。这两天电视，电视新闻节目基本上被游行反抗的声音占领了，真不知道政府会怎么收场。正如一个朋友讲的，不是说好的五十年不变吗，怎么又改主意了呢。
这篇课文告诉我们一个深刻的道理：有时候宁愿挨饿，也要保卫一些东西！#国民教育#
